# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key


In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,13.18,69,100,2.12,AU,1690427982
1,1,lompoc,34.6391,-120.4579,20.08,87,100,5.36,US,1690427813
2,2,saint-joseph,-21.3667,55.6167,18.21,81,79,4.21,RE,1690427983
3,3,san antonio de pale,-1.4014,5.6325,22.84,90,100,5.08,GQ,1690427983
4,4,waitangi,-43.9535,-176.5597,8.23,90,88,0.45,NZ,1690427984


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_city_data= city_data_df.hvplot.points(
    "Lng",
    "Lat",
     size = "Humidity",
     color= "City",
     geo = True,
     tiles = "OSM"
)

# Display the map
map_city_data

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Filter for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & 
    (city_data_df["Max Temp"] <= 30) & 
    (city_data_df["Humidity"] <= 50) & 
    (city_data_df["Cloudiness"] <= 30) & 
    (city_data_df["Wind Speed"] <= 10)
]

# Display the cities with ideal weather conditions
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,yellowknife,62.4560,-114.3525,23.29,35,20,5.66,CA,1690428005
64,64,kone,-21.0595,164.8658,24.27,41,0,4.44,NC,1690428011
121,121,alvorada,-12.4800,-49.1247,23.72,34,18,1.57,BR,1690428035
126,126,aileu,-8.7281,125.5664,24.72,47,4,1.40,TL,1690428041
172,172,isola di capo rizzuto,38.9600,17.0933,21.77,48,0,2.92,IT,1690428056


In [18]:
# Define the criteria
max_temp_criteria = (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30)
humidity_criteria = city_data_df["Humidity"] <= 50

# Filter the DataFrame based on criteria and create a new DataFrame
filtered_cities_df = city_data_df[max_temp_criteria & humidity_criteria].copy()

# Drop rows with null values in the new DataFrame
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,alice springs,-23.7000,133.8833,23.75,46,75,7.20,AU,1690427991
47,47,yellowknife,62.4560,-114.3525,23.29,35,20,5.66,CA,1690428005
64,64,kone,-21.0595,164.8658,24.27,41,0,4.44,NC,1690428011
73,73,klyuchi,52.2667,79.1667,21.93,43,100,1.68,RU,1690428015
85,85,namtsy,62.7161,129.6658,23.84,37,77,2.92,RU,1690428020


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=filtered_cities_df[["City", "Country", "Lat", "Lng","Humidity","Max Temp"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Max Temp,Hotel Name
14,alice springs,AU,-23.7000,133.8833,46,23.75,
47,yellowknife,CA,62.4560,-114.3525,35,23.29,
64,kone,NC,-21.0595,164.8658,41,24.27,
73,klyuchi,RU,52.2667,79.1667,43,21.93,
85,namtsy,RU,62.7161,129.6658,37,23.84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1 # just want first hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address =name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"] #filter
hotel_df

Starting hotel search
alice springs - nearest hotel: Aurora Alice Springs
yellowknife - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
klyuchi - nearest hotel: No hotel found
namtsy - nearest hotel: No hotel found
tannum sands - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
maine soroa - nearest hotel: Maïné-Soroa
nikolayevsk-on-amure - nearest hotel: Север
alvorada - nearest hotel: No hotel found
talnakh - nearest hotel: Талнах
aileu - nearest hotel: Guest House Aileu Villa
chernyshevskiy - nearest hotel: No hotel found
qarqaraly - nearest hotel: Каркаралинск
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
isola di capo rizzuto - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
south hedland - nearest hotel: Port Haven
badger - nearest hotel: Hotel North Pole
jackson - nearest hotel: Brookside
ulladulla - nearest hotel: Hotel Marlin
uliastay - nearest hotel: Болоржин зочид буудал
arenapolis - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Max Temp,Hotel Name
14,alice springs,AU,-23.7000,133.8833,46,23.75,Aurora Alice Springs
64,kone,NC,-21.0595,164.8658,41,24.27,Pacifik Appartels
92,portland,US,45.5234,-122.6762,43,26.08,the Hoxton
104,maine soroa,NE,13.2114,12.0241,46,29.52,Maïné-Soroa
108,nikolayevsk-on-amure,RU,53.1406,140.7300,47,27.34,Север
122,talnakh,RU,69.4865,88.3972,48,26.23,Талнах
126,aileu,TL,-8.7281,125.5664,47,24.72,Guest House Aileu Villa
159,qarqaraly,KZ,49.4058,75.4742,36,25.54,Каркаралинск
166,college,US,64.8569,-147.8028,50,29.77,BEST WESTERN PLUS Pioneer Park Inn
175,udachny,RU,66.4167,112.4000,40,21.92,Вилюй


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
import hvplot.pandas

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",           # Column representing the longitude
    y="Lat",           # Column representing the latitude
    size="Max Temp",   # Size of the points based on Max Temp column
    scale=1,
    color="City",      # Color the points based on the City Name
    alpha=0.5,       
    tiles="OSM",       # Map tile source, OpenStreetMap in this case
    frame_width=800,
    frame_height=500,
    hover_cols=["City", "Max Temp", "Hotel Name", "Hotel Address"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name)